# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st



# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [3]:
employee = pd.read_csv(r'C:\Users\Acer\OneDrive\Desktop\Labs\lab-hypothesis-testing-1\your-code\Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
employee.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
none_values = employee.isnull().sum()
none_values


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
A = employee['Salary or Hourly']

In [7]:
employee['Salary or Hourly'].value_counts()['Salary']


25161

In [8]:
employee['Salary or Hourly'].value_counts()['Hourly']


8022

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [9]:
employee.groupby('Department', as_index=False)['Job Titles'].value_counts

<bound method DataFrameGroupBy.value_counts of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x00000180563C5B50>>

In [10]:
employee.pivot_table(index=['Department'], values=['Job Titles'], aggfunc=['count']).reset_index()

,Department,count
,,Job Titles
0,ADMIN HEARNG,39
1,ANIMAL CONTRL,81
2,AVIATION,1629
3,BOARD OF ELECTION,107
4,BOARD OF ETHICS,8
5,BUDGET & MGMT,46
6,BUILDINGS,269
7,BUSINESS AFFAIRS,171
8,CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [19]:
# H0: hourly wage is greater than $30/hr
# H1: hourly wage is less than $30/hr
alpha = 0.05

In [20]:
h_wage = employee[employee['Hourly Rate'] >= 30]
h_wage

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
19,"ABDULLAH, RASHAD",ELECTRICAL MECHANIC (AUTOMOTIVE),GENERAL SERVICES,F,Hourly,40.0,NaN,46.10
24,"ABERCROMBIE, TIMOTHY",MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
36,"ABRAMS, DANIELLE T",SANITATION LABORER,STREETS & SAN,F,Hourly,40.0,NaN,36.21
...,...,...,...,...,...,...,...,...
33161,"ZURAWSKI, JEFFREY",FRM OF MACHINISTS - AUTOMOTIVE,GENERAL SERVICES,F,Hourly,40.0,NaN,48.85
33164,"ZUREK, FRANCIS",ELECTRICAL MECHANIC,OEMC,F,Hourly,40.0,NaN,46.10
33169,"ZWARYCZ, THOMAS J",POOL MOTOR TRUCK DRIVER,WATER MGMNT,F,Hourly,40.0,NaN,35.60
33174,"ZYGADLO, JOHN P",MACHINIST (AUTOMOTIVE),GENERAL SERVICES,F,Hourly,40.0,NaN,46.35


In [21]:
h_wage = employee[employee['Hourly Rate'] <= 35]
mu = h_wage['Hourly Rate'].mean()
mu

19.36725613802256

In [23]:
sample = h_wage['Hourly Rate']

In [24]:
s = sample.std(ddof=1)
n = len(sample)

In [25]:
stat = (30-mu) / (s/np.sqrt(n))
stat

79.53384531515458

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [ ]:
# Your code here:



Using the `crosstab` function, find the department that has the most hourly workers. 

In [ ]:
# Your code here:



The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [ ]:
# H0: hourly wage is <= 35
# H1: hourly wage is > 35
alpha = 0.05

In [13]:
hr_wage = employee[employee['Hourly Rate'] <= 35]
mu = hr_wage['Hourly Rate'].mean()
mu

19.36725613802256

In [14]:
sample = hr_wage['Hourly Rate']

In [15]:
s = sample.std(ddof=1)

In [16]:
n = len(sample)

In [17]:
stat = (35-mu) / (s/np.sqrt(n))

In [18]:
stat

116.93427851827472

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [ ]:
# Your code here:



# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

